### **Download Sentiment Dataset**

In [ ]:
! pip install --quiet datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which 

In [ ]:
from datasets import load_dataset

sentiment_dataset = load_dataset("rasyosef/amharic-sentiment")
sentiment_dataset

Generating train split:   0%|          | 0/2223 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/279 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
    test: Dataset({
        features: ['clean_tweet', 'label'],
        num_rows: 279
    })
})

In [ ]:
sentiment_dataset["train"].features

{'clean_tweet': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None)}

In [ ]:
label_names = sentiment_dataset["train"].features["label"].names
label_names

['negative', 'positive']

### **Tokenize Tweets**

In [ ]:
from transformers import AutoTokenizer

model_id = "FacebookAI/xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(tokenizer.tokenize("ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ ምዕተ [MASK] ተቆጥሯል።"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

['▁ከ', 'ሀ', 'ገ', 'ራቸው', '▁ከኢትዮጵያ', '▁ከ', 'ወጡ', '▁', 'ግማሽ', '▁', 'ምዕ', 'ተ', '▁[', 'MAS', 'K', ']', '▁ተ', 'ቆ', 'ጥ', 'ሯል።']


In [ ]:
def tokenize_dataset(samples):
  tokenized_samples = tokenizer(samples["clean_tweet"], truncation=True, max_length=512)
  return tokenized_samples

tokenize_dataset({"clean_tweet":["ከሀገራቸው ከኢትዮጵያ ከወጡ ግማሽ"]})

{'input_ids': [[0, 1464, 21608, 5430, 66052, 165627, 1464, 87365, 6, 230446, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
preprocessed_datasets = sentiment_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=["clean_tweet"]
  )
preprocessed_datasets

Map:   0%|          | 0/2223 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

Map:   0%|          | 0/279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 2223
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 279
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 279
    })
})

### **Load Amharic BERT model**

In [ ]:
id2label={i: label for i, label in enumerate(label_names)}
label2id={label: i for i, label in enumerate(label_names)}

print(id2label, label2id)

{0: 'negative', 1: 'positive'} {'negative': 0, 'positive': 1}


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    # device_map="cuda"
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Finetuning**

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
from transformers import TrainingArguments

batch_size = 32
epochs = 8

training_args = TrainingArguments(
    output_dir=model_id+"-finetuned",
    learning_rate=2e-5,
    #lr_scheduler_type="linear",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    seed=16,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  precision = metric2.compute(predictions=predictions, references=labels, average='macro')["precision"]
  recall = metric3.compute(predictions=predictions, references=labels, average='macro')["recall"]
  f1 = metric4.compute(predictions=predictions, references=labels, average='macro')["f1"]

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }

compute_metrics(([[1,0], [0,1]], [0,1]))

{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=preprocessed_datasets["train"],
    eval_dataset=preprocessed_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.69023197889328,
 'eval_accuracy': 0.5376344086021505,
 'eval_precision': 0.26881720430107525,
 'eval_recall': 0.5,
 'eval_f1': 0.34965034965034963,
 'eval_runtime': 5.8098,
 'eval_samples_per_second': 48.023,
 'eval_steps_per_second': 1.549}

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.619000,0.498306,0.806452,0.809385,0.801550,0.803357
2,0.519700,0.412942,0.827957,0.830484,0.823721,0.825516
3,0.413200,0.396822,0.853047,0.872235,0.844341,0.848170
4,0.363900,0.429093,0.838710,0.838344,0.840233,0.838411
5,0.314000,0.449539,0.845878,0.848352,0.842016,0.843824
6,0.271300,0.436699,0.842294,0.841665,0.840853,0.841215
7,0.220800,0.448605,0.853047,0.853895,0.850310,0.851551
8,0.187000,0.473321,0.845878,0.846612,0.843101,0.844310


TrainOutput(global_step=560, training_loss=0.3636075292314802, metrics={'train_runtime': 160.7126, 'train_samples_per_second': 110.657, 'train_steps_per_second': 3.484, 'total_flos': 772105044730500.0, 'train_loss': 0.3636075292314802, 'epoch': 8.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.44860532879829407,
 'eval_accuracy': 0.8530465949820788,
 'eval_precision': 0.8538947478333507,
 'eval_recall': 0.8503100775193799,
 'eval_f1': 0.8515514489274174,
 'eval_runtime': 4.9761,
 'eval_samples_per_second': 56.068,
 'eval_steps_per_second': 1.809,
 'epoch': 8.0}

### **Evaluate Model**

In [ ]:
import torch

num_samples = len(preprocessed_datasets["test"])
inputs = data_collator([preprocessed_datasets["test"][i] for i in range(num_samples)])
# print(inputs)

#output = model(**inputs)
output = model(**{k: v.to("cuda") for k, v in inputs.items()})
y_pred = np.argmax(output.logits.tolist(), axis=-1)
y_test = np.array(preprocessed_datasets["test"]["label"])

print(y_pred)
print(y_test)

[1 1 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 1
 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0
 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 0 0 1 1 1
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 1 1 1 0
 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 1 1 0 1 1 0 1 1 1 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1
 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0]
[1 1 0 1 0 0 1 1 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1
 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0
 0 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 0 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0
 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0
 1 1 1 1 1 0 0 1 0 0 0 0 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 1 0

In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_pred)

array([[129,  21],
       [ 27, 102]])

In [ ]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       150
           1       0.83      0.79      0.81       129

    accuracy                           0.83       279
   macro avg       0.83      0.83      0.83       279
weighted avg       0.83      0.83      0.83       279



### **Test in Pipeline**

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)
pipe("ቆንጆ ፊልም ነው")

[{'label': 'positive', 'score': 0.9494823217391968}]

In [ ]:
pipe("አሪፍ ልብስ")

[{'label': 'positive', 'score': 0.930647611618042}]

In [ ]:
pipe("ጅላንፎ")

[{'label': 'negative', 'score': 0.9695268273353577}]

In [ ]:
pipe("እዚህ ሰፈር በማታ መጓዝ እፈራለሁ")

[{'label': 'positive', 'score': 0.7926247715950012}]

In [ ]:
pipe("ዩክሬን እና ሩስያ ከባድ ውግያ ላይ ናቸው")

[{'label': 'negative', 'score': 0.9581869840621948}]

**Mis-classified**

In [ ]:
misclassified = []
for i in range(len(y_pred)):
  if y_pred[i] != y_test[i]:
    misclassified.append(i)

print(misclassified)

[8, 12, 24, 25, 29, 41, 49, 51, 53, 59, 65, 71, 72, 81, 87, 89, 93, 95, 100, 101, 106, 107, 115, 116, 120, 126, 136, 137, 144, 147, 170, 173, 197, 201, 203, 211, 219, 220, 239, 240, 243, 249, 250, 254, 258, 260, 263, 270]


In [ ]:
for idx in misclassified:
  input_ids = preprocessed_datasets["test"][idx]["input_ids"]
  label = preprocessed_datasets["test"][idx]["label"]
  print(label, tokenizer.decode(input_ids))

0 <s> እንደሱማ ከሆነ እናንተም ቦታ ላይኖራችሁ ነው</s>
1 <s> እንዲህ አይነት ርዕስ ጽፎ ለበዐል ይጠብቁን ድንግርግር ይላል ግን አሻም በበዓልም አልቀልድም ካለ ምን ይደረግ? 😃 ስሜነህ አያሌውና ሱራፌል ሲጠይቁ እንኳን ብንሰማ ይሻለናል በውነት።</s>
1 <s> ዛሬ እንደ እኔ ምህረት የበዛለት ማን ነው???</s>
0 <s> ልክ ብለሃል። እኛማ ተተብትበናል።</s>
0 <s> ያሰብከው ሁሉ ይሳካ! በእውነትና እውቀትና በተመሰረትርው ራዕይ ፓርቲ ኢትዮጵያና ህዝቦችዋ ታፍረውና ተከውረው ይኖራሉ።</s>
0 <s> እውነት እንነጋገር ከተባለ ሹመት ከምንለው ሹፈት ብንልው አይሻልም? አንድ በወታደራዊ ሳይንስ ከፍተኛ ጀነራልን እንዲሁ ከመጣል ካልሆነ በክብር በጡረታ ማግለል አይሻልም? ጥፋተኛ አይደሉም ከተባለ ደሞዛቸውን ከፍሎ ይቅርታ ጠይቆ መመለስ ነበር💚💛❤️</s>
0 <s> በመካከላችን ማንም የለም ፣ በገዛ እጄ ከገደለው ፍጥረት ሁሉ ጋር እኖራለሁ ምንም እንኳን ከእንግዲህ ከእኛ ጋር ባይሆንም</s>
0 <s> ያገኙ የመሰላቸውን ክፍተት በሙሉ እንደ ጥሩ እድል ተጠቅመው እሳት ከማቀበል ወደሗላ አይሉም።</s>
0 <s> ታሪካችን ፡ እንደገና ፡ ተበዉዞ ፡ መስራት ፡ አለበት ፡ እስካሁን ፡ ያለው ፡ የኣንድ ፡ ወገን ፡ ታሪክ ፡ ነው።</s>
0 <s> ከሀገሬው ይልቅ ለቻይናዉያን ምቹ የሆነች ሀገር 🙃...</s>
1 <s> ኦሮምያ የደጋገዎች አገር ናት እንጂ አጋች ታጋች ዲራማ የሚካሄድ በት ክልል አይደለም</s>
0 <s> ነገር ፍለጋ በቃ አሪፍ ሚያስብል ምስሎሽ ነው አይደል መስፍኔ</s>
1 <s> : ጉድለትን ቢያወሩት፣ ለምን ይሄ አልሆነም እያሉ ቢጠይቁ ማለቂያም መልስም አይገኝም። 'እኔ ምን ላድርግ' ብሎ መነሳትን የመሰለ የእርካታ ምንጭ ግን ፍጹም የለም። በርታ 